# Índice Invertido y Modelo Booleano

In [112]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vilch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Tokenizar y eliminar stopwords

In [113]:
with open("stoplist.txt", encoding="utf-8") as file:
    stoplist = [line.rstrip().lower() for line in file]

def preProcesamiento(texto):
    words = []
    # 1- tokenizar
    tokens = nltk.word_tokenize(texto)
    # 2- eliminar signos
    for token in tokens:
        if token.isalnum():
            words.append(token.lower())
    # 3- eliminar stopwords, agregar mas stopwords
    words = [word for word in words if word not in stoplist]
    # 4- Aplicar la reduccion de palabras (lexema)
    # 5- Aplicar la normalizacion de palabras con SnowballStemmer
    stemmer = nltk.SnowballStemmer("spanish")
    words = [stemmer.stem(word) for word in words]

    return words

# test
preprocesedWords = preProcesamiento("hola a todos! aqui programando en python, que divertido es esto!")
print(preprocesedWords)


['hol', 'program', 'python', 'divert']


### Construir Índice Invertido

In [114]:
def indexar(books):    
    indice = dict()
    for file_name in books:
        File = open(file_name, encoding="utf-8")
        texto = File.read().rstrip();
        # 1- preprocesar
        words = preProcesamiento(texto)
        # 2- agregar al indice los keywords a manera de diccionario. solo la repeticion de las palabras por cada document
        for word in words:
            if word in indice:
                if file_name not in indice[word]:
                    indice[word].append(file_name)
            else:
                indice[word] = []
                indice[word].append(file_name)
        
    return indice

# test
books = ["libro1.txt", "libro2.txt", "libro3.txt", "libro4.txt", "libro5.txt", "libro6.txt"]
Indice = indexar(books)
print(sorted(Indice.items()))

[('111º', ['libro1.txt']), ('abaj', ['libro6.txt']), ('abiert', ['libro5.txt']), ('abism', ['libro2.txt', 'libro3.txt']), ('acab', ['libro1.txt', 'libro4.txt', 'libro5.txt', 'libro6.txt']), ('acamp', ['libro5.txt']), ('accidental', ['libro1.txt']), ('accion', ['libro3.txt']), ('acebed', ['libro2.txt']), ('acerc', ['libro2.txt']), ('acompañ', ['libro1.txt', 'libro2.txt', 'libro5.txt', 'libro6.txt']), ('aconsej', ['libro3.txt']), ('acontec', ['libro2.txt', 'libro3.txt']), ('actos', ['libro3.txt']), ('acud', ['libro1.txt', 'libro2.txt', 'libro5.txt', 'libro6.txt']), ('adentr', ['libro1.txt']), ('afortun', ['libro4.txt']), ('agot', ['libro6.txt']), ('agu', ['libro2.txt']), ('aguil', ['libro6.txt']), ('air', ['libro6.txt']), ('alcanz', ['libro2.txt']), ('alde', ['libro1.txt']), ('alegr', ['libro6.txt']), ('alert', ['libro1.txt', 'libro4.txt', 'libro5.txt']), ('alli', ['libro1.txt', 'libro2.txt', 'libro5.txt', 'libro6.txt']), ('altern', ['libro5.txt']), ('amenaz', ['libro4.txt']), ('amig', [

### Aplicar consultas

In [115]:
def AND(list1, list2):
    result = []
    i = j = 0

    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1

    return result

def OR(list1, list2):
    result = []
    i = j = 0

    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            result.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    return result

def AND_NOT(list1, list2):
    result = []

    i = j = 0
    
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            j += 1
    return result

def L(word):
    word = preProcesamiento(word)[0]
    return Indice[word]


In [116]:
result = AND(L("anillo"), L("frodo"))
print("Los libros son:")
print(result)
#['libro1.txt', 'libro2.txt', 'libro4.txt', 'libro6.txt']

Los libros son:
['libro1.txt', 'libro2.txt', 'libro4.txt', 'libro6.txt']


In [117]:
result = OR(L("anillo"), L("frodo"))
print("Los libros son:")
print(result)

Los libros son:
['libro1.txt', 'libro2.txt', 'libro3.txt', 'libro4.txt', 'libro5.txt', 'libro6.txt']


In [118]:
result = AND_NOT(L("frodo"), L("anillo"))
print("Los libros son:")
print(result)

Los libros son:
['libro3.txt', 'libro5.txt']


In [119]:
#CONSULTA 1 con comunidad, frondo and gondor
result = AND_NOT(AND(L("comunidad"), L("frodo")), L("unico"))
print("Los libros son:")
print(result)

#CONSULTA 2 con abismo or acaba and not acampar
result = AND_NOT(OR(L("abismo"), L("acaba")), L("acampar"))
print("Los libros son:")
print(result)

#CONSULTA 3 con acontecimientos and actos or alerta
result = OR(AND(L("acontecimientos"), L("actos")), L("alerta"))
print("Los libros son:")
print(result)



Los libros son:
[]
Los libros son:
['libro1.txt', 'libro2.txt', 'libro3.txt']
Los libros son:
['libro1.txt', 'libro3.txt']
